In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
from merge.preprocessing.tokenizer import AutoTokenizer


tokenizer = AutoTokenizer.create("bert-base-uncased")

In [ ]:
tokenizer = AutoTokenizer.create("bert-base-uncased")

In [ ]:
tokenizer.encode("Hello, my dog is cute")

In [ ]:
fw = load_dataset("roneneldan/TinyStories", split="train", cache_dir="./hf_cache", num_proc=16)


In [5]:
from merge.preprocessing.tokenizer import AutoTokenizer

tokenizer = AutoTokenizer.create("EleutherAI/gpt-neo-125m")
tokenizer.add_special_tokens({"pad_token": "<pad>"})

1

In [7]:
import tokenizers.processors as processors

bos = "<|begin_of_text|>"
tokenizer._tokenizer.post_processor = processors.Sequence(
    [
        processors.ByteLevel(trim_offsets=False),
        processors.TemplateProcessing(
            single=f"{bos}:0 $A:0",
            pair=f"{bos}:0 $A:0 {bos}:1 $B:1",
            special_tokens=[
                (bos, tokenizer.bos_token_id),
            ],
        ),
    ]
)

In [12]:
batch = ["This is a test, what is a test", "This is"]

tokenizer(batch, truncation=True, padding="max_length", max_length=6)

{'input_ids': [[50256, 1212, 318, 257, 1332, 11], [50256, 1212, 318, 50257, 50257, 50257]], 'attention_mask': [[1, 1, 1, 1, 1, 1], [1, 1, 1, 0, 0, 0]]}

In [13]:
tokenizer.decode(50257)

'<pad>'

In [4]:
tokenizer.bos_token_id

50256

In [1]:
from merge.preprocessing.tokenizer import AutoTokenizer

tokenizer = AutoTokenizer.create("microsoft/phi-2")


/Users/leonericsson/projects/github/merge/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2")

tokenizer.bos_token_id

50256

In [14]:
from datasets import load_from_disk

fw = load_from_disk("data/pretokenized_roneneldan_TinyStories")

In [9]:
from merge.modules.attention import causal_attention_mask

mask = causal_attention_mask(512)

type(mask)

torch.Tensor

In [3]:
from merge.modules.attention import causal_attention_mask
from merge.architectures.llama import Llama, LlamaConfig

model_config = LlamaConfig(
    vocab_size=50257,
    seq_len=512,
    d_model=256,
    hidden_dim=512,
    num_heads=8,
    num_kv_heads=4,
    num_layers=6,
    rope_theta=0.1,
    norm_eps=1e-6,
    activation_fn="silu",
    mlp_bias=False,
    mlp_dropout=0.0,
    attention_bias=False,
    attention_dropout=0.0,
    pos_encoding_type="rope",
    mlp="mlp_swiglu",
    normalization="rmsnorm",
    attention="gqa",
)

In [4]:
model = Llama(model_config)

causal mask: None


In [5]:
from merge.modules.transformer import Transformer

model_t = Transformer(model_config)

model_t.causal_mask

tensor([[[[ 1.0000e+00, -1.0000e+04, -1.0000e+04,  ..., -1.0000e+04,
           -1.0000e+04, -1.0000e+04],
          [ 1.0000e+00,  1.0000e+00, -1.0000e+04,  ..., -1.0000e+04,
           -1.0000e+04, -1.0000e+04],
          [ 1.0000e+00,  1.0000e+00,  1.0000e+00,  ..., -1.0000e+04,
           -1.0000e+04, -1.0000e+04],
          ...,
          [ 1.0000e+00,  1.0000e+00,  1.0000e+00,  ...,  1.0000e+00,
           -1.0000e+04, -1.0000e+04],
          [ 1.0000e+00,  1.0000e+00,  1.0000e+00,  ...,  1.0000e+00,
            1.0000e+00, -1.0000e+04],
          [ 1.0000e+00,  1.0000e+00,  1.0000e+00,  ...,  1.0000e+00,
            1.0000e+00,  1.0000e+00]]]])

In [5]:
model.causal_mask

In [27]:
import torch
import torch.nn.functional as F

# Let's simulate a vocabulary where:
# pad_token_id = 0
# a_token_id = 1
# b_token_id = 2
# vocab_size = 3

# Original sequence: "aab" padded to length 5
original = torch.tensor([[1, 1, 2, 0, 0, 0]])  # [a, a, b, pad, pad]
input_ids = original[:, :-1]  # [a, a, b, pad, pad]
target_ids = original[:, 1:] # [a, b, pad, pad, pad]



In [41]:
input_ids, target_ids

(tensor([[1, 1, 2, 0, 0]]), tensor([[1, 2, 0, 0, 0]]))

In [28]:
# Create fake logits - we'll make them "perfect" predictions
# Shape: (batch=1, seq=5, vocab=3)
logits = torch.zeros(1, 5, 3)
# For each position, make the correct prediction have highest probability
logits[0, 0, 1] = 10.0  # predict 'a' at position 0
logits[0, 1, 2] = 10.0  # predict 'b' at position 1
# We don't care about the predictions for padded positions

# Test both with and without ignore_index
criterion_with_ignore = torch.nn.CrossEntropyLoss(ignore_index=0)
criterion_without_ignore = torch.nn.CrossEntropyLoss()

In [29]:
# Reshape for loss calculation
batch, seq, vocab = logits.shape
logits_reshaped = logits.reshape(batch * seq, vocab)
target_ids_reshaped = target_ids.reshape(batch * seq)

loss_with_ignore = criterion_with_ignore(logits_reshaped, target_ids_reshaped)
loss_without_ignore = criterion_without_ignore(logits_reshaped, target_ids_reshaped)

print(f"Loss with ignore_index=0: {loss_with_ignore.item()}")
print(f"Loss without ignore_index: {loss_without_ignore.item()}")

Loss with ignore_index=0: 9.083335316972807e-05
Loss without ignore_index: 0.6592037081718445


In [31]:
logits_reshaped.shape, target_ids_reshaped.shape

(torch.Size([5, 3]), torch.Size([5]))

In [32]:
loss_with_ignore

tensor(9.0833e-05)